In [1]:
!pip install py-entitymatching

     |████████████████████████████████| 2.0MB 12.7MB/s 
     |████████████████████████████████| 788kB 49.2MB/s 
     |████████████████████████████████| 1.1MB 50.3MB/s 
     |████████████████████████████████| 368kB 54.7MB/s 
     |████████████████████████████████| 665kB 52.0MB/s 
  Created wheel for py-entitymatching: filename=py_entitymatching-0.4.0-cp37-cp37m-linux_x86_64.whl size=2634154 sha256=5dd6cc306a1d56e23717e1c2bd3ba4268f9c2d7c193167eccfc5fdaa592f3be2
  Stored in directory: /root/.cache/pip/wheels/88/65/50/a8bc0f26e8a82c931658153c4f37cbf5debc22cb0f61dbcd05
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3698113 sha256=c39e37cecdfa6144be61c6c8f173d07e9d634b783142aee408e04ece1a8cef5d
  Stored in directory: /root/.cache/pip/wheels/58/30/8e/6fa463100ff6e8595b131067ec43a118c607f621ac297e7dad
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055279 sha256=a77c6ba6a4f583f4d4

In [1]:
# https://hpi.de/naumann/projects/repeatability/datasets/restaurants-dataset.html
# http://www.iesl.cs.umass.edu/datasets.html - CORA dataset
# Importing the libraries
import sys
import pandas as pd
import os
import math
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
path_variable = "/content/gdrive/My Drive/BTP/Code/Dataset/Music/"
method = "/content/gdrive/My Drive/BTP/Code/Method3/weights_music/"

In [3]:
import py_entitymatching as em
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
pandas version: 1.1.5
magellan version: 0.4.0


In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = path_variable+"music1.csv"
path_B = path_variable+"music2.csv"
A = em.read_csv_metadata(path_A, key='Sno')
B = em.read_csv_metadata(path_B, key='Sno')

# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


Number of tuples in A: 6907
Number of tuples in B: 55923
Number of tuples in A X B (i.e the cartesian product): 386260161


In [5]:
import pickle
import numpy as np
import torch

dbfile = open(path_variable+'files/feature_matrix', 'rb')    
cand_file = open(path_variable+'files/C', 'rb')  
S = pickle.load(dbfile)
cand = pickle.load(cand_file)

S = torch.tensor(S.values,dtype=torch.double)
S = S[:,3:]
# S = S.to_numpy(dtype=np.longdouble)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initial weights would be 1/(num_attributes*num_sim_scores) due to add-to-1 constraint
w_init  = (1 / S.shape[1] )
# W = np.full((S.shape[0], S.shape[1]), w_init,dtype=np.longdouble)
W = torch.DoubleTensor(S.shape[1],1).fill_(w_init)

In [6]:
# verify values in S
print(S.shape)

d1 = dict()
d2 = dict()

t1 = A.to_numpy()
t2 = B.to_numpy()

for x in t1:
  if(x[-1] in d1):
    d1[x[-1]]+=1
  else:
    d1[x[-1]]=1

for x in t2:
  if(x[-1] in d2):
    d2[x[-1]]+=1
  else:
    d2[x[-1]]=1

print("d1 : ",d1)
print("d2 : ",d2)

torch.Size([12829, 28])
d1 :  {'Country': 1367, 'Pop': 1758, 'Other': 538, 'Dance': 952, 'Alternative': 484, 'Soundtrack': 397, 'Electronic': 211, 'Rock': 515, 'Hip-Hop': 685}
d2 :  {'Dance': 8274, 'Other': 13135, 'Electronic': 4019, 'Soundtrack': 3594, 'Country': 8317, 'Rock': 3248, 'Pop': 7130, 'Alternative': 4806, 'Hip-Hop': 3400}


In [7]:
# ith index of genre_groups denotes genre of ith record pair
genre_groups = []
for i in cand['ltable_Group']:
    genre_groups.append(i)
genre_indices = {}
n = len(genre_groups)

for i in genre_groups:
    genre_indices[i] = []

for i in range(len(genre_groups)):
    genre_indices[genre_groups[i]].append(i)

In [14]:
# ith row of RLS matrix will return RLS value of the ith record pair 
# return nx1 matrix, where n=number of record pairs 
def get_RLS(W,S):
    rls = torch.matmul(S,torch.square(W))
    return rls

# returns groupwise rls_values dictionary
def get_RLS_i(W,S,genre_indices):
    rls_vector = get_RLS(W,S) # nx1
    rls_values_i = {}
    for genre in genre_indices:
        rls_values_i[genre] = 0
    for genre in genre_indices:
        indices = genre_indices[genre]
        for i in indices:
            rls_values_i[genre] += float(rls_vector[i])
    return rls_values_i

############################################################################################

def get_precomputed_values(W,S,phi,tau):
    numerator = torch.exp(phi*get_RLS(W,S)) - torch.exp(phi* (tau-get_RLS(W,S)))
    # print(numerator)
    denominator = torch.exp(phi*get_RLS(W,S)) + torch.exp(phi* (tau-get_RLS(W,S)))
    return torch.divide(numerator,denominator)

def derivative_loss_function(W,S,j,precomputed_values):
    # print(S[:,j].reshape(-1,1).shape)
    # print(torch.transpose(S[:,j],0,1).shape)
    # print(precomputed_values.shape)
    m = torch.matmul(torch.transpose(S[:,j].reshape(-1,1),0,1),precomputed_values)
    der = -2 * W[j,0] * m
    return der

############################################################################################

def get_sval_i(S,j,indi):
    val = 0
    # print(S.shape)
    for x in indi:
      val += S[x,j]
    return val

def get_fairness_precomputed_value(W,S,type_indices):
    rls_values_sum_dic = get_RLS_i(W,S,type_indices)
    num = np.array([0]*W.shape[0])
    rls_sum_all = torch.sum(get_RLS(W,S)) # sigma RLS_all

    for type_ in type_indices:
        rls_sum_i = rls_values_sum_dic[type_]
        pi =  rls_sum_i / rls_sum_all
        fi = float(len(type_indices[type_])/n)

        for j in range(W.shape[0]):
          num[j] += (fi/pi) * (rls_sum_all * get_sval_i(S,j,type_indices[type_]) - rls_sum_i*torch.sum(S[:,j])) 
    
    return num / float((rls_sum_all*rls_sum_all)) 

############################################################################################


# Here j is the jth weight
def get_fairness_derivative(W,S,j,precomputed_value_fairness):
    # s1 = torch.transpose(S[j,:].reshape(-1,1),0,1).shape
    # w1 = torch.transpose(W[j,:].reshape(-1,1),0,1).shape


    return -2 * W[j,:] * precomputed_value_fairness[j]


############################################################################################

def algorithm_gradient_descent(W,S,learning_rate,iterations,phi,tau,lambdaa):

    for i in range(iterations):
        W_new = torch.zeros(W.shape) 
        # print(W.shape)
        # print(W_new.shape)

        sum_W_new = 0
        precomputed_values_em = get_precomputed_values(W,S,phi,tau)
        precomputed_value_fairness = get_fairness_precomputed_value(W,S,genre_indices)
        
        
        for j in range(W.shape[0]):
            # if j==W.shape[0]-1:
            #     print(get_fairness_derivative(W,S,j,precomputed_value_fairness))

            loss_value = derivative_loss_function(W,S,j,precomputed_values_em) + lambdaa * get_fairness_derivative(W,S,j,precomputed_value_fairness)
            # print(get_fairness_derivative(W,S,j,precomputed_value_fairness))
            # print(derivative_loss_function(W,S,j,precomputed_values_em))
            # print("jay")
            # break
            W_new[j,0] = W[j,0] - learning_rate * loss_value
            sum_W_new += W_new[j,0]

        for j in range(W.shape[0]):
            W_new[j,0] = W_new[j,0] / float(sum_W_new)
        
        W = W_new.double()
        # print(W)
        
        if((i+1)%10==0):
          print("Iteration " + str(i+1) + " completed ")        

    return W

In [9]:
######### Skew metric adopted from https://arxiv.org/pdf/1905.01989.pdf (refer this paper for the metric)

def fair_calc(weights,S,flag,k):
  
  rls_matrix = get_RLS(weights,S)
  cand['score'] = rls_matrix.numpy()
  candy = cand.sort_values(by=['score'], ascending=False)

  # print(candy)
  # k=500

  d3 = dict()
  d4 = dict()

  temp = candy.to_numpy()
  cnt = 0

  for _id,pair in enumerate(temp):
    # print(_id)
    # print(pair,pair[11],pair[20])
    if(_id>k):
      # print(_id)
      break

    if(pair[11] in d3):
      d3[pair[11]]+=1
    else:
      d3[pair[11]]=1
      
    if(pair[20] in d4):
      d4[pair[20]]+=1
    else:
      d4[pair[20]]=1
    
    if(pair[11]==(pair[20])):
      cnt+=1

  
  # print("d3 : ", d3)
  # print("d4 : ",d4)

  # print(cnt)
  
  mini = -1
  maxi = 1
  d_skew = {}
  flagy = 0
  d_avg = 0
  for d in d1.keys():
    if(d in d3.keys()):
      d_skew[d] = np.log( d3[d]*len(t1)/(d1[d]*k))
      d_avg += np.abs(d_skew[d])
      if(flagy==0):
        flagy+=1
        mini = d_skew[d]
        maxi = d_skew[d]
      
      mini = min(mini,d_skew[d])
      maxi = max(maxi,d_skew[d])
    # else:
    #   print(d)
  
  if(flag):
    print("Group distribution in result : ",d3)
    print("Skew metric max-min: ", maxi-mini)
    print("Skew metric avg: ", d_avg/len(d1.keys()))
 
  return maxi-mini

In [10]:
#No learning
print(W)
fair_calc(W,S,True,500)

tensor([[0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357],
        [0.0357]], dtype=torch.float64)
Group distribution in result :  {'Alternative': 111, 'Dance': 35, 'Country': 246, 'Rock': 79, 'Other': 5, 'Hip-Hop': 7, 'Pop': 12, 'Electronic': 6}
Skew metric max-min:  3.514470723052824
Skew metric avg:  1.199843245246169


3.514470723052824

In [15]:
#No tweak of weights for fairness
# weights corresponding to lambda = 0 

weights0 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,0)
print(weights0)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[1.3304e-09],
        [1.9245e-09],
        [6.8237e-02],
        [7.0599e-06],
        [1.6387e-22],
        [1.2139e-22],
        [4.4591e-07],
        [4.4668e-17],
        [4.9202e-12],
        [2.8343e-16],
        [1.1077e-01],
        [1.4631e-09],
        [1.7408e-12],
        [1.1078e-10],
        [7.6622e-02],
        [9.5729e-09],
        [2.9951e-05],
        [1.1243e-02],
        [6.4664e-06],
        [1.5495e-02],
        [4.8944e-08],
        [9.3562e-06],
        [2.3510e-10],
        [1.4736e-03],
        [3.5623e-16],
        [3.2228e-03],
        [7.4530e-09],
        [7.1288e-01]], dtype=torch.float64)


In [16]:
weights1 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,500)
print(weights1)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[1.3904e-09],
        [2.0173e-09],
        [7.3136e-02],
        [7.7369e-06],
        [2.3459e-22],
        [1.7284e-22],
        [4.9575e-07],
        [6.1912e-17],
        [4.3801e-12],
        [1.6841e-16],
        [1.1824e-01],
        [1.4228e-09],
        [1.7949e-12],
        [1.1991e-10],
        [8.6991e-02],
        [1.0945e-08],
        [3.1107e-05],
        [1.2269e-02],
        [6.9859e-06],
        [1.6858e-02],
        [6.8108e-08],
        [1.1900e-05],
        [3.3452e-10],
        [1.8282e-03],
        [4.3458e-16],
        [3.4290e-03],
        [8.5781e-09],
        [6.8718e-01]], dtype=torch.float64)


In [17]:
# weights corresponding to lambda= 1000 
weights2 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,1000)
print(weights2)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[1.0092e-09],
        [1.4577e-09],
        [6.7135e-02],
        [6.2271e-06],
        [1.3412e-22],
        [9.8058e-23],
        [4.1073e-07],
        [3.9996e-17],
        [3.7071e-12],
        [1.2376e-16],
        [1.0402e-01],
        [1.2845e-09],
        [1.1726e-12],
        [7.8445e-11],
        [7.5391e-02],
        [7.8786e-09],
        [2.3797e-05],
        [1.0110e-02],
        [5.2581e-06],
        [1.3890e-02],
        [5.5677e-08],
        [1.0350e-05],
        [2.6058e-10],
        [1.6577e-03],
        [3.2773e-16],
        [3.2991e-03],
        [7.6142e-09],
        [7.2445e-01]], dtype=torch.float64)


In [18]:
# weights corresponding to lambda= 5000 
weights3 = algorithm_gradient_descent(W,S,1e-4,100,-100,1,5000)
print(weights3)

Iteration 10 completed 
Iteration 20 completed 
Iteration 30 completed 
Iteration 40 completed 
Iteration 50 completed 
Iteration 60 completed 
Iteration 70 completed 
Iteration 80 completed 
Iteration 90 completed 
Iteration 100 completed 
tensor([[8.1161e-10],
        [6.1034e-10],
        [5.8543e-02],
        [6.7277e-06],
        [4.0959e-24],
        [2.7623e-24],
        [1.5213e-07],
        [3.2994e-18],
        [1.7141e-11],
        [2.5705e-16],
        [1.3742e-01],
        [7.4132e-09],
        [6.1136e-13],
        [3.4687e-11],
        [7.0453e-02],
        [4.9630e-09],
        [1.9141e-05],
        [9.4432e-03],
        [4.0769e-06],
        [1.3352e-02],
        [2.7686e-08],
        [6.6297e-06],
        [1.0024e-10],
        [1.3435e-03],
        [5.4886e-17],
        [2.5018e-03],
        [3.5872e-09],
        [7.0690e-01]], dtype=torch.float64)


In [19]:
import pickle

dbfile = open(method+'w0', 'wb') 
pickle.dump(weights0, dbfile) 
dbfile.close()

dbfile = open(method+'w1', 'wb') 
pickle.dump(weights1, dbfile) 
dbfile.close()

dbfile = open(method+'w2', 'wb') 
pickle.dump(weights2, dbfile) 
dbfile.close()

dbfile = open(method+'w3', 'wb') 
pickle.dump(weights3, dbfile) 
dbfile.close()


In [20]:
import pickle

dbfile = open(method+'w0', 'rb') 
weights0 = pickle.load(dbfile)
dbfile.close()

dbfile = open(method+'w1', 'rb') 
weights1 = pickle.load(dbfile)
dbfile.close()

dbfile = open(method+'w2', 'rb') 
weights2 = pickle.load(dbfile)
dbfile.close()

dbfile = open(method+'w3', 'rb') 
weights3 = pickle.load(dbfile)
dbfile.close()

In [21]:
def comp_overlap(w1,w2,k):
  rls_matrix = get_RLS(w1,S)
  cand['score'] = rls_matrix.numpy()
  candy1 = cand.sort_values(by=['score'], ascending=False)

  # print(candy)

  dw1 = dict()
  temp1 = candy1.to_numpy()
  
  rls_matrix = get_RLS(w2,S)
  cand['score'] = rls_matrix.numpy()
  candy2 = cand.sort_values(by=['score'], ascending=False)

  # print(candy)
  dw2 = dict()
  temp2 = candy2.to_numpy()
  overlp = 0
  
  # print(candy1)
  # print(candy2)

  # print(temp1[0][1],temp1[0][2])
  # print(temp1)
  # print(temp2)
  # return 
  dname = {}

  
  for _i,i in enumerate(temp1):
    if(_i>k-1):
      break
    dname[str(i[1])+"#"+str(i[2])] = 1
  
  # print(len(dname))

  for _j,j in enumerate(temp2):
      if(_j>k-1):
        break
      if(str(j[1])+"#"+str(j[2]) in  dname.keys()):
        overlp+=1
  
  return overlp


In [22]:


path_G = path_variable+"music_labeled_data.csv"
G = em.read_csv_metadata(path_G, 
                        key='_id',
                        ltable=A, rtable=B, 
                        fk_ltable='ltable.Sno', fk_rtable='rtable.Sno')
print(len(G))

gdic = {}
cnt_no_g = 0
cnt_g = 0

G1 = G.to_numpy()
for i,x in enumerate(G1):
  s = str(x[2])+"#"+str(x[3])  
  s1 = str(x[3])+"#"+str(x[2])
  # print(s)
  gdic[s] = x[-3]
  gdic[s1] = x[-3]
  if(x[-3]==1):
    cnt_g+=1
  else:
    cnt_no_g+=1
print("Positive in ground truth",cnt_g)
print("Negative in ground truth",cnt_no_g)

print(gdic.keys())

def check_gt(w1,S,k):
  rls_matrix = get_RLS(w1,S)
  cand['score'] = rls_matrix.numpy()
  candy1 = cand.sort_values(by=['score'], ascending=False)
  temp1 = candy1.to_numpy()
  
  overlp = 0
  neg = 0

  for _j,j in enumerate(temp1):
      if(_j>k-1):
        break
      s = str(j[1])+"#"+str(j[2])

      if(s in  gdic.keys()):
        # print(s)
        if(gdic[s]==1):
          overlp+=1
        else:
          neg+=1

  return overlp,neg

Metadata file is not present in the given path; proceeding to read the csv file.


539
Positive in ground truth 132
Negative in ground truth 407
dict_keys(['111#53124', '53124#111', '148#50767', '50767#148', '206#41214', '41214#206', '211#19812', '19812#211', '250#53111', '53111#250', '250#53124', '53124#250', '251#53007', '53007#251', '252#53004', '53004#252', '252#53084', '53084#252', '253#53033', '53033#253', '253#53073', '53073#253', '254#53033', '53033#254', '255#49141', '49141#255', '262#53045', '53045#262', '263#53053', '53053#263', '265#53087', '53087#265', '266#53007', '53007#266', '272#53067', '53067#272', '276#52996', '52996#276', '277#53056', '53056#277', '283#53027', '53027#283', '290#22340', '22340#290', '292#22310', '22310#292', '303#22342', '22342#303', '304#22316', '22316#304', '304#22317', '22317#304', '307#22330', '22330#307', '311#22331', '22331#311', '312#22335', '22335#312', '314#22313', '22313#314', '314#22341', '22341#314', '318#22310', '22310#318', '323#22319', '22319#323', '326#22338', '22338#326', '331#22312', '22312#331', '338#26133', '261

In [23]:
# print(S.shape)
# print()

for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights0,S,True,i)    #lambda = 0


print("####################################################### \n")
for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights1,S,True,i)    #lambda = 500
  print("Overlap with base result = "+str(comp_overlap(weights0,weights1,i))+" /"+ str(i))
  print("Overlap with ground truth weights_this", check_gt(weights1,S,i))


print("####################################################### \n")
for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights2,S,True,i)    #lambda = 1000  
  print("Overlap with base result = "+str(comp_overlap(weights0,weights2,i))+" /"+ str(i))
  print("Overlap with ground truth weights_this", check_gt(weights2,S,i))


print("####################################################### \n")
for i in [250,500,750,1000]:
  print("\n"+str(i))
  fair_calc(weights3,S,True,i)    #lambda = 5000
  print("Overlap with base result = "+str(comp_overlap(weights0,weights3,i))+" /"+ str(i))
  print("Overlap with ground truth weights_this", check_gt(weights3,S,i))



250
Group distribution in result :  {'Soundtrack': 1, 'Alternative': 49, 'Other': 1, 'Country': 135, 'Rock': 59, 'Dance': 6}
Skew metric max-min:  4.121229103403768
Skew metric avg:  1.174067475862527

500
Group distribution in result :  {'Soundtrack': 1, 'Alternative': 93, 'Other': 1, 'Country': 272, 'Rock': 70, 'Dance': 46, 'Pop': 12, 'Electronic': 6}
Skew metric max-min:  4.673286789733832
Skew metric avg:  1.4819720653241235

750
Group distribution in result :  {'Soundtrack': 1, 'Alternative': 98, 'Other': 2, 'Country': 350, 'Rock': 82, 'Dance': 107, 'Pop': 93, 'Electronic': 18}
Skew metric max-min:  4.6214955984467005
Skew metric avg:  1.1107471841050343

1000
Group distribution in result :  {'Soundtrack': 1, 'Alternative': 98, 'Other': 38, 'Country': 384, 'Rock': 82, 'Dance': 119, 'Pop': 250, 'Electronic': 29}
Skew metric max-min:  4.714204996550968
Skew metric avg:  0.675479615653273
####################################################### 


250
Group distribution in result :  

In [ ]:
def get_results_em(weights):
  # Load the pre-labeled data
  rls_scores_G = get_RLS(weights,G_features)

  # make seperate column for predicted label
  count_pred = 0
  G['pred_label'] = 0
  for i in range(rls_scores_G.shape[0]):
      if float(rls_scores_G[i,0]) > 0.2:
          G['pred_label'][i] = 1
          count_pred+=1
      else:
          G['pred_label'][i] = 0
  print("\n")
  print("percentage of labels=1: " , count_pred/rls_scores_G.shape[0])

  print()
  print("--------------Evaluation summary---------------")
  # Evaluate the predictions
  eval_result = em.eval_matches(G, 'label', 'pred_label')
  em.print_eval_summary(eval_result)

In [ ]:
print(W)
get_results_em(W)

In [ ]:
get_results_em(weights1)

In [ ]:
get_results_em(weights2)